In [6]:
import gensim
import os
import re
import numpy as np
import json as js
import argparse
import pandas as pd
import sklearn
import joblib
import torch
import matplotlib.pyplot as plt
import datasets
import tensorflow as tf
from tqdm import tqdm
from datasets import Dataset, DatasetDict
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.manifold import Isomap
from sklearn.naive_bayes import GaussianNB
from gensim.models import KeyedVectors
from transformers import BertTokenizer, TFBertModel
from transformers import AutoTokenizer



2024-05-27 15:44:14.473630: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 15:44:14.664480: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 15:44:15.430452: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained("bert-base-uncased")



2024-05-27 15:44:25.131351: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-27 15:44:25.135072: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transfo

In [8]:
def getDataJSON(route):
    with open(route,"r",encoding="utf-8") as f:
        result = [js.loads(jline.replace("diff","value")) for jline in f.read().splitlines()]
    return result

In [20]:
def specialTokens(text):
    text=text.replace("\n", "")
    marked_text = "[CLS] " + text + " [SEP]"
    #marked_text = marked_text.split()
    return marked_text

In [11]:
def tokenization(example,label):

    return tokenizer(example[label],padding=True, truncation=True, max_length=512,return_tensors="pt")

In [12]:
def utils_bert_embedding(txt, tokenizer, model):
    '''
    Word embedding with Bert (equivalent to nlp["word"]).
    :parameter
        :param txt: string 
        :param tokenizer: transformers tokenizer
        :param nlp: transformers bert
    :return
        tensor sentences x words x vector (1x3x768) 
    '''
    # tokenize sentence to tokens (integers)
    idx = tokenizer.encode(txt,truncation=True,max_length=512)
    # convert to array of shape (1, num_words+2) - EOS and CLS added
    idx = np.array(idx)[None,:]
    # generate embeddings for each token - output is a tuple
    embedding = model(idx)
    # select first member of the tuple, remove first dimension which is 1 to get (num_words,embedding size 712)
    # exclude CLS and EOS tokens
    X = np.array(embedding[0][0])
    return X

In [21]:
def dataToCsv(datadir):
    aux = os.path.join(datadir, "train.jsonl")
    data = pd.DataFrame(getDataJSON(aux)).set_index("id")
    data[['text1','text2']] = pd.DataFrame(data.pair.tolist(), index= data.index)

    aux = os.path.join(datadir, "train_truth.jsonl")

    del data["pair"]
    data2 = pd.DataFrame(getDataJSON(aux)).set_index("id")
    data = pd.merge(data,data2,how='outer',left_index=True,right_index=True)
    del data2

    X_0 = data["text1"]
    X_1 = data["text2"]
    X = X_0 + "[SEP]" + X_1
    X = X.apply(specialTokens)

    Y = data["value"].to_frame()

    X_series = pd.concat([X, Y], axis=1)
    X_series.to_csv(datadir + "prepared.csv")
    
    
    


In [22]:
data=dataToCsv("../corpus/dataset1/train/prepared/")

In [23]:
data[0][0]

/tmp/ipykernel_114607/154174995.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data[0][0]


"[CLS] I'm not arguing with you here, I'm simply trying to contextualize this for you. To the extent that they are there, it is with your consent. The state has passed laws making sure that vulnerable people (not saying he's one) don't get abused (not saying you're abusing him), and in casting a wide net to save as many vulnerable little birds as possible from hitting the floor after being kicked out of their nest wrongfully, the state has (as much from a lack of better options as from any other reason) created a circumstance where occasionally some not-so-vulnerable little bird can take advantage of someone else's nest.[SEP]He's at my place half the time and his fiance(e)'s place the other half of the time. He's been (homeless) couch surfing for several years and only recently got engaged to his other partner. We don't have any current issues that would lead me to want this arrangement to stop, but I do want to protect my own legal rights. [SEP]"

In [7]:

def BERTProcessing(datadir,tokenizer,model):
    aux = os.path.join(datadir, "train.jsonl")
    data = pd.DataFrame(getDataJSON(aux)).set_index("id")
    data[['text1','text2']] = pd.DataFrame(data.pair.tolist(), index= data.index)

    aux = os.path.join(datadir, "train_truth.jsonl")

    del data["pair"]
    data2 = pd.DataFrame(getDataJSON(aux)).set_index("id")
    data = pd.merge(data,data2,how='outer',left_index=True,right_index=True)
    del data2

    
    X_0 = data["text1"]
    X_0 = X_0.apply(specialTokens)
    X_0_frame=X_0.to_frame()
    lst_mean_vecs_X_0 = [(utils_bert_embedding(txt,tokenizer, model)[0]) for txt in tqdm(X_0_frame["text1"])]

    X_1 = data["text2"]
    X_1 = X_1.apply(specialTokens)
    X_1_frame=X_1.to_frame()
    lst_mean_vecs_X_1 = [(utils_bert_embedding(txt,tokenizer, model)[0]) for txt in tqdm(X_1_frame["text2"])]

    list_X_0 = np.array(lst_mean_vecs_X_0)
    list_X_1 = np.array(lst_mean_vecs_X_1)

    Y = data["value"].to_frame()

    X_df0=pd.DataFrame(list_X_0,index=data.index)
    X_df1=pd.DataFrame(list_X_1,index=data.index)
    
    
 
    Y = data["value"]
    print("$$$$$$-DONE-$$$$$$")
    return X_df0,X_df1,Y



In [8]:
def comparation(X1, X2):
    result=X1 - X2
    return result

In [16]:
data=BERTProcessing("../corpus/dataset1/train/prepared/",tokenizer,model)
data
def save_series_to_jsonl(series, filename):
    series.to_json(filename, orient='records', lines=True)

def load_series_from_jsonl(filename):
    return pd.read_json(filename, lines=True)

def row_to_vector(row):
    return row.values

X_vectors0=data[0].apply(row_to_vector,axis=1)
X_vectors1=data[1].apply(row_to_vector,axis=1)
X_resta=comparation(X_vectors0,X_vectors1)

X = pd.concat([X_resta, data[2]], axis=1)

save_series_to_jsonl(X, '../corpus/dataset1/train/processed/dataset1.jsonl')

###########################

data=BERTProcessing("../corpus/dataset2/train/prepared/",tokenizer,model)
data

def row_to_vector(row):
    return row.values

X_vectors0=data[0].apply(row_to_vector,axis=1)
X_vectors1=data[1].apply(row_to_vector,axis=1)
X_resta=comparation(X_vectors0,X_vectors1)

X = pd.concat([X_resta, data[2]], axis=1)

save_series_to_jsonl(X, '../corpus/dataset2/train/processed/dataset2.jsonl')

###########################

data=BERTProcessing("../corpus/dataset3/train/prepared/",tokenizer,model)
data

def row_to_vector(row):
    return row.values

X_vectors0=data[0].apply(row_to_vector,axis=1)
X_vectors1=data[1].apply(row_to_vector,axis=1)
X_resta=comparation(X_vectors0,X_vectors1)

X = pd.concat([X_resta, data[2]], axis=1)

save_series_to_jsonl(X, '../corpus/dataset3/train/processed/dataset3.jsonl')



100%|██████████| 12904/12904 [38:08<00:00,  5.64it/s]


$$$$$$-DONE-$$$$$$


100%|██████████| 28216/28216 [1:19:06<00:00,  5.94it/s]


$$$$$$-DONE-$$$$$$


100%|██████████| 19113/19113 [52:58<00:00,  6.01it/s] 


$$$$$$-DONE-$$$$$$


: 

In [9]:
def load_series_from_jsonl(filename):
    return pd.read_json(filename, lines=True)

In [21]:
# Cargar la serie desde el archivo .jsonl
X = load_series_from_jsonl('../corpus/dataset3/train/processed/dataset3.jsonl')
X

/tmp/ipykernel_77792/3734992399.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  return pd.read_json(filename, lines=True)
/tmp/ipykernel_77792/3734992399.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  return pd.read_json(filename, lines=True)
/tmp/ipykernel_77792/3734992399.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'

,0,value
0,"[0.7014677525, 0.5042597055, 0.066472709200000...",1
1,"[-0.45516845580000004, -0.5592068434, 0.093394...",0
2,"[0.4117831886, 0.4675021172, -0.030528605, -0....",0
3,"[-0.2600257993, 0.0054365098, 0.33509260420000...",0
4,"[0.1627482623, 0.0485873818, -0.3508000374, -0...",0
...,...,...
19108,"[-0.3444256485, -0.2093059719, -0.282564282400...",1
19109,"[-0.09240455930000001, -0.0842291713, 0.408709...",1
19110,"[0.3342903256, 0.25868806240000003, 0.12647709...",0
19111,"[-0.1530846357, -0.18637441100000002, -0.03203...",1


In [18]:
X["0"]

0        [0.26854580640000003, 0.1192571223, 0.48951399...
1        [0.18595962230000002, 0.2051506341, -0.0806472...
2        [-0.2992926836, 0.0093523264, -0.5162963867, 0...
3        [-0.2105169296, -0.27284821870000003, -0.11649...
4        [-0.1370931119, 0.4245643318, 0.31885379550000...
                               ...                        
28211    [-0.13925941290000002, 0.110483706, -0.2658059...
28212    [0.4680920839, 0.011689554900000001, 0.2339281...
28213    [-0.123173587, -0.2438404858, -0.1423244923, 0...
28214    [-0.1027491614, 0.10870979730000001, 0.0940337...
28215    [0.2684149742, 0.2917635143, 0.282811820500000...
Name: 0, Length: 28216, dtype: object

In [12]:
def cosine_sim(X1,X2):
    cos=np.dot(X1, X2) / (np.linalg.norm(X1) * np.linalg.norm(X2))
    return cos

In [30]:
Cosine = pd.Series(X.apply(lambda x: cosine_sim(x[0], x[1]), axis=1))

In [31]:
type(Cosine)

pandas.core.series.Series

In [28]:
type(data[2])

pandas.core.series.Series

In [11]:

def plot_points_with_labels(vectors_df, labels_df,titulo):
  # Asegúrate de que las etiquetas y los vectores tengan el mismo índice
  # Separa los vectores según las etiquetas
    vectors_label_0 = vectors_df[labels_df == 0]
    vectors_label_1 = vectors_df[labels_df == 1]
    
  # Crea un gráfico de puntos
    plt.figure(figsize=(8, 6))
    plt.scatter(vectors_label_1.iloc[:, 0], vectors_label_1.iloc[:, 1], color='red', label='Con cambio')
    plt.scatter(vectors_label_0.iloc[:, 0], vectors_label_0.iloc[:, 1], color='blue', label='Sin cambio')
    
    
  # Configura las etiquetas y el título
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(titulo)
   
  # Muestra la leyenda
    plt.legend()
   
  # Muestra el gráfico
    plt.show()

In [63]:
def redimensionarPCA(X):
    pca = PCA(n_components=2)
    X_vectores = X.apply(pd.Series)
    pca.fit(X_vectores.T)
    X_PCAvec = pd.DataFrame(pca.components_.T,index=X_vectores.index)
    return X_PCAvec


In [23]:
plot_points_with_labels(Cosine,data[2],"Cosine Sim")

IndexError: single positional indexer is out-of-bounds

<Figure size 800x600 with 0 Axes>

In [82]:
X_1=redimensionarPCA(X)
plot_points_with_labels(X_1,data[2],"PCA")

AttributeError: 'numpy.ndarray' object has no attribute 'apply'

In [156]:
def redimensionarTSNE(X):
    tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=1000)
    X_vectores = X.apply(pd.Series)
    X_vec = tsne.fit_transform(X_vectores)
    X_TSNEvec = pd.DataFrame(X_vec,index=X_vectores.index)
    print(X_TSNEvec)
    return X_TSNEvec

In [131]:
def redimensionarISOMAP(X):
    isomap=Isomap(n_components=2)
    X_vectores = X.apply(pd.Series)
    X_vec = isomap.fit_transform(X_vectores)
    X_ISOMAPvec = pd.DataFrame(X_vec,index=X_vectores.index)
    print(X_ISOMAPvec)
    return X_ISOMAPvec

In [40]:
def SVM(X,Y,exitmodel):
  print('%%%%%%TRAINING MODEL%%%%%%')
  X_train=np.array(X.tolist())
  Y_train=np.array(Y.tolist())
  svm_model = SVC(kernel='rbf')
  svm_model.fit(X_train, Y_train)
  joblib.dump(svm_model,exitmodel)

In [39]:
def SVM_linear(X,Y,exitmodel):
  print('%%%%%%TRAINING MODEL%%%%%%')
  X_train=np.array(X.tolist())
  X_train=X_train.reshape(-1, 1)
  Y_train=np.array(Y.tolist())
  svm_model = SVC(kernel='linear')
  svm_model.fit(X_train, Y_train)
  joblib.dump(svm_model,exitmodel)

In [68]:
def NaiveBayes(X,Y,exitmodel):
  print('%%%%%%TRAINING MODEL%%%%%%')
  X_train=np.array(X.tolist())
  Y_train=np.array(Y.tolist())
  Bayes_model = GaussianNB()
  Bayes_model.fit(X_train, Y_train)
  joblib.dump(Bayes_model,exitmodel)

In [12]:
def RNA(X,Y,exitmodel):
  print('%%%%%%TRAINING MODEL%%%%%%')
  X_train=np.array(X.tolist())
  Y_train=np.array(Y.tolist())
  modelRNA = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(768,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')])

  modelRNA.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
  modelRNA.fit(X_train,Y_train, epochs=10, batch_size=32)
  return modelRNA
  #joblib.dump(modelRNA,exitmodel)



In [22]:
modelRNA = RNA(X["0"],X["value"],"./model/dataset3/RNA_model.pkl")

%%%%%%TRAINING MODEL%%%%%%
Epoch 1/10


/home/gianni/Documentos/DamianW2V/Transformers/env1/lib/python3.10/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


598/598 ━━━━━━━━━━━━━━━━━━━━ 1s 794us/step - accuracy: 0.5509 - loss: 0.6873
Epoch 2/10
598/598 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step - accuracy: 0.6343 - loss: 0.6401
Epoch 3/10
598/598 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step - accuracy: 0.6804 - loss: 0.5923
Epoch 4/10
598/598 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step - accuracy: 0.7456 - loss: 0.5163
Epoch 5/10
598/598 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - accuracy: 0.7928 - loss: 0.4482
Epoch 6/10
598/598 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step - accuracy: 0.8462 - loss: 0.3557
Epoch 7/10
598/598 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step - accuracy: 0.8893 - loss: 0.2728
Epoch 8/10
598/598 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step - accuracy: 0.9240 - loss: 0.2000
Epoch 9/10
598/598 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step - accuracy: 0.9462 - loss: 0.1475
Epoch 10/10
598/598 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step - accuracy: 0.9561 - loss: 0.1194


In [14]:
def load_model(model_path):
  model = joblib.load(model_path)
  return model

In [15]:
def prepare_test_data(datavalidationdirectory):
  data_parsed = []

  for element in [x for x in os.listdir(datavalidationdirectory) if '.txt' in x]:
    id = re.findall(r'^[a-zA-Z\-0-9]*\-([0-9]+).txt$',element)[0]
    with open(os.path.join(datavalidationdirectory,f'problem-{id}.txt'),'r', encoding="utf-8") as f:
      data = f.readlines()

  
    comb = []
    for i in range(0,len(data)-1):
      comb.append({"id":f'{id}-{i+1}-{i+2}',"pair": [data[i],data[i+1]]})
    data_parsed.append(comb)

  with open(os.path.join(datavalidationdirectory,'validation.json'),'w',encoding="utf-8") as f:
    js.dump(data_parsed, f, indent=4)

  print("validation data prepared")


In [16]:
def evaluate_model(datavalidation_path,prediction_path,modelSVM):

  with open(os.path.join(datavalidation_path,'validation.json')) as f:
    data = js.load(f)
  print("----------- PREDICTION ------------")
  for dict in data:
    conf = []
    for dict_line in dict:
      X_0 = np.array(utils_bert_embedding(specialTokens(dict_line["pair"][0]),tokenizer, model)[0] )
      X_1 = np.array(utils_bert_embedding(specialTokens(dict_line["pair"][1]),tokenizer, model)[0] )
      X = X_0 - X_1
      #X = np.absolute(X)
      X = X.reshape(1,-1)
      temp = np.round(modelSVM.predict(X))
      print(temp)
      conf.append(int(temp))
      temp=[]

      problem_number = int(re.findall(r'([0-9]+)-[0-9]+-[0-9]+',dict_line["id"])[0])
      print(os.path.join(prediction_path,f"solution-problem-{problem_number}.json"))
      with open(os.path.join(prediction_path,f"solution-problem-{problem_number}.json"),"w+") as f:
        js.dump({"changes":conf},f)

In [43]:
modelo=("./model/dataset1/CosineSim_model_linear.pkl")

In [49]:
prepare_test_data("../corpus/dataset3/validation/")

validation data prepared


In [23]:
evaluate_model("../corpus/dataset3/validation/","../corpus/dataset3/predictionsBERT/",modelRNA)

----------- PREDICTION ------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[[0.]]
../corpus/dataset3/predictionsBERT/solution-problem-796.json


/tmp/ipykernel_77792/2028244608.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  conf.append(int(temp))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[0.]]
../corpus/dataset3/predictionsBERT/solution-problem-796.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[0.]]
../corpus/dataset3/predictionsBERT/solution-problem-796.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[1.]]
../corpus/dataset3/predictionsBERT/solution-problem-796.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[0.]]
../corpus/dataset3/predictionsBERT/solution-problem-796.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
[[1.]]
../corpus/dataset3/predictionsBERT/solution-problem-796.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[0.]]
../corpus/dataset3/predictionsBERT/solution-problem-728.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[1.]]
../corpus/dataset3/predictionsBERT/solution-problem-728.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[0.]]
../corpus/dataset3/predictionsBERT/solution-problem-728.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[0.]]
../corpus/dataset3/predictionsBERT/solution-problem-728.json
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[0.]]
.